### Librerías

In [1]:
import geopandas as gpd
import pandas as pd
from Valor_catastral_construccion import *
from Valor_catastral_terreno import *
from calculo_predial import *
import random
import numpy as np 


### Base de datos

In [2]:

get_factor_edad_va()

0.89211

In [2]:
path_hector = r"G:\Mi unidad\Equipo_Agua\Geo\Data\Prueba\Muestra_modelo_final.xlsx"
# datos = pd.read_csv(path_hector, encoding='utf8')
datos = pd.read_excel(path_hector)
datos.fillna(0,inplace=True)
datos.columns

Index(['Unnamed: 0', 'Indice_gdf', 'index__Cru', 'Tipo calle', 'Nombre Cal',
       'Zona_Banda', 'Manzanas_B', 'COD_BandaV', 'Val m2_Ban', 'Pseudo_cla',
       'Zona_Manz', 'Manzanas_M', 'COD_Manz', 'Tipo', 'Frente', 'Fondo',
       'Area_AH', 'Val m2_Man', 'Pseudo_c_1', 'mun', 'zona', 'manz', 'cve_cat',
       'key', 'Lon', 'Lat', 'Tipo_cruce', 'Tipo_cru_1', 'clase_dete', 'conf',
       'Area_Model', 'Medio', 'Popular', 'Interes', 'Campestre', 'Rural',
       'NivelesM_H', 'NivelesM_1', 'NivelesM_2', 'NivelesP_H', 'NivelesP_1',
       'NivelesIS_', 'NivelesI_1', 'NivelesC_H', 'NivelesR_H', 'Nivel_mini',
       'AlturaM_HM', 'AlturaM__1', 'AlturaM__2', 'AlturaP_HP', 'AlturaP__1',
       'AlturaIS_H', 'AlturaIS_1', 'AlturaC_HC', 'AlturaR_HR', 'Altura_min',
       'Medida_lx', 'Medida_ly', 'geometry', 'Esquinero|Intermedio'],
      dtype='object')

In [3]:
# Obtenemos las columnas de altura
cols_altura = np.array([])
for col in datos:
    if col.lower().strip().startswith('altura'):
        cols_altura = np.append(cols_altura, col)
cols_altura

array(['AlturaM_HM', 'AlturaM__1', 'AlturaM__2', 'AlturaP_HP',
       'AlturaP__1', 'AlturaIS_H', 'AlturaIS_1', 'AlturaC_HC',
       'AlturaR_HR', 'Altura_min'], dtype='<U32')

### Llenado de datos faltantes

In [8]:
datos['Frente_base'] = datos['Frente']/100
datos['Fondo_base'] = datos['Fondo']/100
datos['Area_base'] = datos['Area_AH']/1000
datos['posicion'] = np.random.choice(['Interior','Intermedio','Esquinero','Frentes_no_contiguos','Cabecero','Manzanero'], len(datos))
datos['grado_conservacion'] = np.random.choice(['Bueno','Normal','Regular','Malo','Ruinoso'], len(datos))
datos['Area_inscrita'] = datos['Area_Model'].map(lambda x: float(x)*random.uniform(0.65,1))

### Calculo de factores

In [21]:
# Valor catastral del terreno
datos['Factor_posicion']      = datos['Esquinero|Intermedio'].fillna(0).map(lambda x: get_factor_posicion(x))
datos['Terreno_Posicion']     = datos['Factor_posicion'].map(lambda x: 'Intermedio' if x==0.5 else 'otro')
datos['factor_frente']        = pd.Series(map(factor_frente,datos['Medida_lx'],datos['Terreno_Posicion'] ))
datos['factor_fondo']         = pd.Series(map(factor_fondo,datos['Medida_ly'], datos['Fondo_base'],datos['Terreno_Posicion']))
for col in cols_altura:
    datos['Factor_Topografia_'+col] = pd.Series(map(factor_topografia,datos[col],datos['Medida_ly'],datos['Terreno_Posicion']))

datos['factor_irregularidad'] = pd.Series(map(factor_irregularidad,datos['Area_Model'], datos['Area_inscrita'], datos['Terreno_Posicion']))
datos['factor_area']          = pd.Series(map(factor_area,datos['Area_Model'], datos['Area_base'],datos['Terreno_Posicion']))
datos['factor_restriccion']   = pd.Series(map(factor_restriccion,datos['Area_Model'], datos['Area_Model'],datos['Terreno_Posicion']))

In [ ]:
# Valor catastral de la construcción
datos['factor_grado_conservacion'] = pd.Series(map(factor_grado_conservacion,datos['grado_conservacion']))
datos['factor_edad'] =0.6 # pd.Series(map(factor_edad,datos['Edad'], datos['Coeficiente_demerito']))
datos['factor_numero_niveles'] = pd.Series(map(factor_numero_niveles,datos['Nivel_minimo']))

### Calculo de valores catastrales

In [25]:
datos.columns
cols_valor_uni = ['Val m2_Man','Val m2_Ban']
cols_factor_top = ['Factor_Topografia_AlturaM_HM','Factor_Topografia_AlturaM__1', 'Factor_Topografia_AlturaM__2',
       'Factor_Topografia_AlturaP_HP', 'Factor_Topografia_AlturaP__1','Factor_Topografia_AlturaIS_H', 
       'Factor_Topografia_AlturaIS_1','Factor_Topografia_AlturaC_HC', 'Factor_Topografia_AlturaR_HR',
       'Factor_Topografia_Altura_min']

In [ ]:
cols_using = ['Area_Model','factor_frente','factor_fondo', ]

In [26]:
for col_vu in cols_valor_uni:
    i = 0
    name = col_vu.split('_')[-1]
    for col_ft in cols_factor_top:
        datos['Valor_catastral_terreno_'+name+'_'+str(i)] = datos['Area_Model']*datos[col_vu]*datos['factor_frente']*datos['factor_fondo']*datos['factor_irregularidad']*datos['factor_area']*datos[col_ft]*datos['factor_posicion']*datos['factor_restriccion']
        i += 1

TypeError: can't multiply sequence by non-int of type 'float'

In [22]:
datos['Valor_catastral_terreno'] = datos['Area_Model']*datos['Val m2_Manz']*datos['factor_frente']*datos['factor_fondo']*datos['factor_irregularidad']*datos['factor_area']*datos['factor_topografia']*datos['factor_posicion']*datos['factor_restriccion']

KeyError: 'Val m2_Manz'

### Exportación de datos

In [ ]:
datos.to_excel(r'C:\Users\ruben\Downloads\Modelo_prueba_v1.xlsx')